In [1]:
!pip install pyops

In [ ]:
import pyops
import logging
import json

In [2]:
# read config file first
with open('config.json', 'r') as f:
    config = json.load(f)
print(config)

{'PEPS_url': 'https://peps.cnes.fr/resto/api/collections/', 'PEPS_collections': ['S1', 'S2', 'S2ST', 'S3'], 'PEPS_formats': ['json', 'atom'], 'COP_url': 'https://scihub.copernicus.eu/dhus/search?q=', 'COP_user': '', 'COP_pwd': '', 'PAVICS_url': 'https://pavics.ouranos.ca/twitcher/ows/proxy/catalog/wps'}


# PEPS
Les descripteurs Opensearch de PEPS sont:
"https://peps.cnes.fr/resto/api/collections/{collection}/describe.xml"

In [3]:
parameters = {
    'resto_tileid' :  "31TCJ" #,
   # '{eo:cloudCover}' : {"value": '[10,70]'}
}
collection = "S2ST"

In [4]:
def PEPS_descr_url(collection):
    descr_URL = config['PEPS_url']
    if collection in config['PEPS_collections']:
      descr_URL += collection
    descr_URL += "/describe.xml"
    return descr_URL

In [5]:
descr_PEPS = PEPS_descr_url(collection)
print(descr_PEPS)

https://peps.cnes.fr/resto/api/collections/S2ST/describe.xml


In [22]:
logging.basicConfig(filename='example.log',level=logging.INFO)
# Client params
# specify either search_endpoint or description_xml_url
# type = 'collection' or 'results'
client = pyops.Client(description_xml_url=descr_PEPS, type='collection')
# search params: force_HTTPS=True, params={}, auth=()


In [27]:
# dir(client)

In [8]:
client.search_params

{'searchTerms': {'clean_tag': 'searchTerms',
  'full_tag': '{searchTerms?}',
  'type': 'text',
  'title': 'Free text search'},
 'productIdentifier': {'clean_tag': 'productIdentifier',
  'full_tag': '{productIdentifier?}',
  'type': 'text',
  'title': 'Product identifier'},
 'count': {'clean_tag': 'count',
  'full_tag': '{count?}',
  'type': 'text',
  'title': 'Number of results returned per page (default 20, max 500)',
  'minInclusive': '1',
  'maxInclusive': '500'},
 'startIndex': {'clean_tag': 'startIndex',
  'full_tag': '{startIndex?}',
  'type': 'text',
  'title': 'Index of the first result returned',
  'minInclusive': '1'},
 'startPage': {'clean_tag': 'startPage',
  'full_tag': '{startPage?}',
  'type': 'text',
  'title': 'Results page to return',
  'minInclusive': '1'},
 'language': {'clean_tag': 'language',
  'full_tag': '{language?}',
  'type': 'text',
  'title': 'Two letters language code according to ISO 639-1',
  'pattern': '^[a-z]{2}$'},
 'geo_uid': {'clean_tag': 'geo:uid',

In [10]:
client.search_param_names

{'q': 'searchTerms',
 'productIdentifier': 'productIdentifier',
 'maxRecords': 'count',
 'index': 'startIndex',
 'page': 'startPage',
 'lang': 'language',
 'identifier': 'geo_uid',
 'geometry': 'geo_geometry',
 'box': 'geo_box',
 'location': 'geo_name',
 'lon': 'geo_lon',
 'lat': 'geo_lat',
 'radius': 'geo_radius',
 'startDate': 'time_start',
 'completionDate': 'time_end',
 'parentIdentifier': 'eo_parentIdentifier',
 'productType': 'eo_productType',
 'processingLevel': 'eo_processingLevel',
 'platform': 'eo_platform',
 'instrument': 'eo_instrument',
 'resolution': 'eo_resolution',
 'organisationName': 'eo_organisationName',
 'orbitDirection': 'eo_orbitDirection',
 'orbitNumber': 'eo_orbitNumber',
 'sensorMode': 'eo_sensorMode',
 'cloudCover': 'eo_cloudCover',
 'snowCover': 'eo_snowCover',
 'cultivatedCover': 'resto_cultivatedCover',
 'desertCover': 'resto_desertCover',
 'floodedCover': 'resto_floodedCover',
 'forestCover': 'resto_forestCover',
 'herbaceousCover': 'resto_herbaceousCover

In [28]:
client = pyops.Client(description_xml_url=descr_PEPS, type='collection')

param_names = client.search_param_names
params = client.search_params
input_params = {}

form_fields = {"tileid":  "31TCJ", "page": "1", 'maxRecords': '50', 'cloudCover': '[10,70]'}
for ff in form_fields:
    value = form_fields[ff]
    if value and ff in param_names:
        print(params[param_names[ff]]["full_tag"])
        input_params[params[param_names[ff]]["full_tag"]] = {"value": value}

# Get Raw Results
raw_results = client.search(params=input_params)
print("total number of results:", client.pagination['total_results'])
print("last page:", client.pagination['last'] )
form_fields = client.pagination['next']
while True:
    print(form_fields)
    for ff in form_fields:
        value = form_fields[ff]
        if value and ff in param_names:
            input_params[params[param_names[ff]]["full_tag"]] = {"value": value}
    client = pyops.Client(description_xml_url=descr_PEPS, type='collection')
    new_raw_results = client.search(params=input_params)
    print(len(new_raw_results))
    assert raw_results[0] != new_raw_results[0]
    raw_results = new_raw_results
    print("last page", client.pagination['last'] )
    
    if form_fields == client.pagination['last'] :
        break
    form_fields = client.pagination['next']
    input_params = {}

print(raw_results)


{resto:tileid?}
{startPage?}
{count?}
{eo:cloudCover?}
https://peps.cnes.fr/resto/api/collections/S2ST/search.atom?maxRecords=50&page=1&cloudCover=[10,70]&tileid=31TCJ
total number of results: 209
last page: {'maxRecords': '50', 'page': '5', 'cloudCover': '%5B10%2C70%5D', 'tileid': '31TCJ'}
{'maxRecords': '50', 'page': '2', 'cloudCover': '%5B10%2C70%5D', 'tileid': '31TCJ'}
https://peps.cnes.fr/resto/api/collections/S2ST/search.atom?maxRecords=50&page=2&cloudCover=%5B10%2C70%5D&tileid=31TCJ
50
last page {'maxRecords': '50', 'page': '5', 'cloudCover': '%5B10%2C70%5D', 'tileid': '31TCJ'}
{'maxRecords': '50', 'page': '3', 'cloudCover': '%5B10%2C70%5D', 'tileid': '31TCJ'}
https://peps.cnes.fr/resto/api/collections/S2ST/search.atom?maxRecords=50&page=3&cloudCover=%5B10%2C70%5D&tileid=31TCJ
50
last page {'maxRecords': '50', 'page': '5', 'cloudCover': '%5B10%2C70%5D', 'tileid': '31TCJ'}
{'maxRecords': '50', 'page': '4', 'cloudCover': '%5B10%2C70%5D', 'tileid': '31TCJ'}
https://peps.cnes.fr/res

In [43]:
entry_fields = client.get_available_fields()
filtered_results = client.filter_entries([{
    "tag": "{http://www.w3.org/2005/Atom}title",
    "name": "title"}, {
    "tag": "{http://www.w3.org/2005/Atom}summary",
    "name": "summary"}])
for results in filtered_results:
    for res in results:
        print(res)

{'tag': '{http://www.w3.org/2005/Atom}title', 'name': 'title', 'attrs': {}, 'text': 'S2A_MSIL1C_20200918T152631_N0209_R068_T20UPD_20200918T171831', 'children': []}
{'tag': '{http://www.w3.org/2005/Atom}summary', 'name': 'summary', 'attrs': {'type': 'text'}, 'text': 'S2A/MSI acquired on 2020-09-18T15:26:31.024Z', 'children': []}
{'tag': '{http://www.w3.org/2005/Atom}title', 'name': 'title', 'attrs': {}, 'text': 'S2A_MSIL1C_20200918T152631_N0209_R068_T19UGR_20200918T171831', 'children': []}
{'tag': '{http://www.w3.org/2005/Atom}summary', 'name': 'summary', 'attrs': {'type': 'text'}, 'text': 'S2A/MSI acquired on 2020-09-18T15:26:31.024Z', 'children': []}
{'tag': '{http://www.w3.org/2005/Atom}title', 'name': 'title', 'attrs': {}, 'text': 'S2A_MSIL1C_20200918T152631_N0209_R068_T20TMR_20200918T171831', 'children': []}
{'tag': '{http://www.w3.org/2005/Atom}summary', 'name': 'summary', 'attrs': {'type': 'text'}, 'text': 'S2A/MSI acquired on 2020-09-18T15:26:31.024Z', 'children': []}
{'tag': '{